In [78]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
import ast
import torch.nn.functional as F
import pandas as pd
#plotting
import matplotlib.pyplot as plt

import data_utils
import models
import importlib
import transformer_models
importlib.reload(data_utils)
importlib.reload(models)
importlib.reload(transformer_models)
from data_utils import *
from models import *
from transformer_models import * 
import torch.optim as optim
from torch.utils.data import DataLoader
import datetime

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.Size([40, 80])
torch.Size([80, 40])
Using device: cuda


In [79]:
from pymatgen.core.periodic_table import Element
MAX_ATOM = 80
# List of atomic numbers
atomic_numbers = list(range(1, MAX_ATOM))

# Dictionary to store atomic radii
atomic_radii = []

# Iterate over the atomic numbers and retrieve atomic radii
for atomic_number in atomic_numbers:
    element = Element.from_Z(atomic_number)
    atomic_radius = element.atomic_radius
    atomic_radii.append(atomic_radius)

#replace None entries with 0 
atomic_radii = [elem if elem != None else 0 for elem in atomic_radii ]

atomic_radii = np.array(atomic_radii)

from pymatgen.core.periodic_table import Element

# Get a list of all element symbols
element_symbols = [elem.symbol for elem in Element]

In [80]:
train_graphs = torch.load('data/train.pt')
train_df = pd.read_csv('data/train.csv')

In [81]:
train_df['atomic_numbers'] = train_df['atomic_numbers'].apply(ast.literal_eval)
atomic_num_list = [np.array(sublist) for sublist in list(train_df['atomic_numbers'])]
indices_to_exclude = [i for i, val in enumerate(atomic_num_list) if np.any(val > MAX_ATOM-1)]
filtered_df = train_df.drop(indices_to_exclude)

In [82]:
atomic_num_list = [np.array(sublist) for sublist in list(filtered_df['atomic_numbers'])]
spacegroup_list = torch.tensor(list(filtered_df['spacegroup.number']))

all_atom_types = [np.concatenate([vec, np.zeros(25 - len(vec))]) for vec in atomic_num_list]
all_atom_types = torch.tensor(np.stack(all_atom_types)).long()
training_data_onehot = torch.nn.functional.one_hot(all_atom_types, num_classes=MAX_ATOM).float()

In [83]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        """
        Args:
            inputs (Tensor): The input tensor of shape (n, 25, 40)
            targets (Tensor): The target tensor of shape (n, 1)
        """
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]


In [84]:
model = ElemFormer().to('cuda')

In [85]:
data_dir = "data/"
dataObject = xrdData("data/", device)

dataObject.make_datasets(1, composition_embedding="compositionseq")
# Create DataLoaders for train and validation sets
train_loader = DataLoader(dataObject.torch_datasets['train'], batch_size=256, shuffle=True)
valid_loader = DataLoader(dataObject.torch_datasets['val'], batch_size=256, shuffle=False)  

In [86]:
# data_dir = "data/"
# dataObject = xrdData("data/", device)

# # Create DataLoaders for train and validation sets
# train_loader = DataLoader(CustomDataset(training_data_onehot[:-1000], spacegroup_list[:-1000]), batch_size=256, shuffle=True, pin_memory=True)
# val_loader = DataLoader(CustomDataset(training_data_onehot[1000:], spacegroup_list[1000:]), batch_size=256, shuffle=True, pin_memory=True)

In [87]:
# Create the model instance and move it to the selected device
output_dim = 230  # Output dimension
ConvModel = XRD_C_SymNet(in_channels=1, output_dim=output_dim, composition_model= None).to(device)
token_size = 50 #dimension of the tokens 
TransModel = TransformerModel(ntoken = output_dim, d_model = token_size, nhead = 10, d_hid=50, nlayers=1, dropout = 0.5).to(device)
model = TransModel
# Define optimizer and loss function
weight_decay = 0  # Example value, adjust based on your needs
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

KeyboardInterrupt: 

In [ ]:
# Define optimizer and loss function
weight_decay = 0  # Example value, adjust based on your needs
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [ ]:
#instantiate experimental data simulator 
simulator = ExperimentalSimulation(device, crop_start=2000, crop_stop = 2000, noise_range = 0.4, drop_width = 500, drop_freq = 2) 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(ConvModel)

31369900

In [ ]:
count_parameters(TransModel)

148156370

In [ ]:
count_parameters(ConvModel) / count_parameters(TransModel)

0.21173507423271776

In [ ]:
max_epochs = 200
metrics = ["accuracy", "loss"]

log = {
    f"{type}": {f"{metric}" : np.zeros(max_epochs) for metric in metrics} for type in ['train', 'val']     
}

for epoch in range(max_epochs):
    model.train()  # Set the model to training mode
    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for xrd, composition, targets in tqdm(train_loader): 
        #xrd = simulator.sim(xrd)
        #xrd = F.normalize(xrd, p=2, dim=1)
        optimizer.zero_grad()
        outputs = model(xrd, composition)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()

    train_accuracy = 100 * correct_train / total_train
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_valid_loss = 0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():  # No gradients needed for validation
        for xrd, composition, targets in tqdm(valid_loader):
            #xrd = simulator.sim(xrd)
            #xrd = F.normalize(xrd, p=2, dim=1)
            outputs = model(xrd, composition)
            loss = criterion(outputs, targets)
            total_valid_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 
            total_valid += targets.size(0)
            correct_valid += (predicted == targets).sum().item()

    valid_accuracy = 100 * correct_valid / total_valid

    total_train_loss = total_train_loss / len(train_loader)
    validation_loss = total_valid_loss / len(valid_loader)

    print(f"Epoch {epoch+1}, Training Loss: {total_train_loss}, Training Accuracy: {train_accuracy}%, Validation Loss: {validation_loss}, Validation Accuracy: {valid_accuracy}%")

    log['train']['accuracy'][epoch] = (train_accuracy)
    log['train']['loss'][epoch] =(total_train_loss)

    log['val']['accuracy'][epoch] =(valid_accuracy)
    log['val']['loss'][epoch] =(validation_loss)

100%|██████████| 16/16 [00:00<00:00, 19.56it/s]


Epoch 1, Training Loss: 3.744630541404088, Training Accuracy: 17.31045751633987%, Validation Loss: 3.844312906265259, Validation Accuracy: 18.35836354220611%


 10%|█         | 12/120 [00:02<00:23,  4.68it/s]


KeyboardInterrupt: 

In [161]:
test_sequence  = torch.ones((1, 2, 5)).to(device)
test_sequence[:, 1, :2] = 2

In [162]:
test_sequence

tensor([[[1., 1., 1., 1., 1.],
         [2., 2., 1., 1., 1.]]], device='cuda:0')

In [163]:
result = transformer_encoder(test_sequence)